*Verified by Leila Belabbassi (to work with Pangeo), July 12, 2018*

#### Note: This note book is a work in progress - There is a problem with the OpenDAP server behavior where the data live. 

In [1]:
import requests
import re
import os
import xarray as xr

In [2]:
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/friedrich.knuth@rutgers.edu/20171207T161702-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample/catalog.html'
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i) for i in x]

In [4]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=20)
cluster

In [5]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://10.40.21.3:43092 Dashboard: /user/leilabbb/proxy/34706/status,Cluster Workers: 12 Cores: 24 Memory: 48.00 GB


In [6]:
%%time
ds = xr.open_mfdataset(datasets, decode_times=False, chunks={'obs': 10000}, parallel=True)

distributed.utils - ERROR - ('open_dataset-aaaaca3d-d918-4d13-b377-10143bcee2f0', 'tcp://10.40.24.4:43648')
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/distributed/utils.py", line 238, in f
    result[0] = yield make_coro()
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.6/site-packages/distributed/client.py", line 1378, in _gather
    traceback)
  File "/opt/conda/lib/python3.6/site-packages/six.py", line 693, in reraise
    raise value
distributed.scheduler.KilledWorker: ('open_dataset-aaaaca3d-d918-4d13-b377-10143bcee2f0', 'tcp://10.40.24.4:43648')


KilledWorker: ('open_dataset-aaaaca3d-d918-4d13-b377-10143bcee2f0', 'tcp://10.40.24.4:43648')

In [ ]:
ds = ds.swap_dims({'obs': 'time'})
ds